In [ ]:
# 有问题的
#  20210715_1627_w1 20210715_1636_w2 20210715_1648_w4

#  20210715_1659_w1w: Items of 'xdata' vector must satisfy the condition: x1 < x2 < ... < xN

# 1-测试是否有bug

In [ ]:
import os
import time
from Lib.YamlFrames import OpenYaml, GetWormName,Just_Get_Raw,Get_First_Frames,Get_End_Frames,Extract_Yaml_Multiprocess_one

time_start = time.time()

workpath = '/home/data2/WormPoseData/ProcessingData/Temp/20210715_1627_w1'

wormname = workpath.split('/')[-1]
print("Extract:",wormname)
# 改变这行填入你的Yaml文件地址
yaml_path = os.path.join(workpath,wormname,wormname+'.yaml')



ListFile = OpenYaml(yaml_path) # 导入数据

ListFile[-1]
linenumber = ListFile[-99].split(':')[-1]

begin_num1 = Get_First_Frames(ListFile)
end_num = Get_End_Frames(ListFile)
All_Frames_num = end_num-begin_num1+1  # 总的帧数
print("Total frames:",All_Frames_num)

# YamlData = Just_Get_Raw(yaml_path)
YamlData = Extract_Yaml_Multiprocess_one(yaml_path)

time_end = time.time()
print('time cost:',time_end-time_start,'s')



# 2-修bug

In [ ]:
# 有问题的
#  20210715_1627_w1 20210715_1636_w2 20210715_1648_w4

#  20210715_1659_w1w: 
# Items of 'xdata' vector must satisfy the condition: x1 < x2 < ... < xN

# 20210715_1627_w1 少帧
# 20210715_1648_w4 最后一帧少几行

import os
import time

workpath = r'H:\WenLabData\LPJ\wen1120 pre 20210715\YamlFiles\20210715_1659_w1w.yaml'

wormname = workpath.split('\\')[-1]
wormname = wormname.split('.')[0]
print("Extract:",wormname)
# 改变这行填入你的Yaml文件地址
yaml_path = os.path.join(workpath)

def OpenYaml(yaml_path:str):
    Yaml_file = []
    with open(yaml_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.rstrip('\n') # 删除末尾的回车
            line = line.strip() # 删除前面的空格
            Yaml_file.append(line)
    return(Yaml_file)

ListFile = OpenYaml(yaml_path) # 导入数据



In [ ]:
import math
def Get_Frames_line(lines):
    # 获取'Frames: ' 行数
    import re
    for i in range(30):
        if re.match('Frames',lines[i]):
            break
    return(i)

def Get_First_Frames(lines):
    linenumber = Get_Frames_line(lines)+2
    FramsNumber = lines[linenumber].split(':')[-1]
    return(int(FramsNumber))

def Get_End_Frames(lines):
    # 获取最后一个Frames的开始一行
    import re
    for i in range(1,100):
        if re.match('FrameNumber',lines[-i]):
            break
    linenumber = lines[-i].split(':')[-1]
    return(int(linenumber))

begin_num1 = Get_First_Frames(ListFile)
end_num = Get_End_Frames(ListFile)
All_Frames_num = end_num-begin_num1+1  # 总的帧数

framesline =  Get_Frames_line(ListFile)  # Frames的行数
real_frames = math.ceil((len(ListFile)-(framesline+1))/99)
print("real frames:",real_frames)
print("End- begin Total frames:",All_Frames_num)

In [ ]:
class YamlFrames(object):
    def __init__(self, name: str, framessize: int):
        import numpy as np

        self.worm_length = 0
        self.name = name
        self.ExperimentTime = 'Sun May 16 20:51:33 2021'
        self.DefaultGridSizeForNonProtocolIllum = np.zeros(
            (1, 2))  # DefaultGridSizeForNonProtocolIllum
        # internal frame number, not nth recorded frame
        self.FrameNumber = np.zeros((framessize, 1))
        # time since start of experiment (in s) = sElapsed+ 0.001*msRemElapsed
        self.TimeElapsed = np.zeros((framessize, 1))
        # N*2*100 x,y BoundaryA position in pixels on camera
        self.BoundaryA = np.zeros((framessize, 2, 100))
        # N*2*100 x,y BoundaryB position in pixels on camera
        self.BoundaryB = np.zeros((framessize, 2, 100))
        # N*2*100 x,y centerline position in pixels on camera
        self.Centerline = np.zeros((framessize, 2, 100))
        self.Head = np.zeros((framessize, 2))  # position in pixels on camera
        self.Tail = np.zeros((framessize, 2))  # position in pixels on camera
        self.DLPisOn = np.zeros((framessize, 1))  # bool whether DLP is active
        # flood light overrides all other patterns and hits entire fov
        self.FloodLightIsOn = np.zeros((framessize, 1))
        # whether pattern is inverted (invert has precedence over floodlight)
        self.IllumInvert = np.zeros((framessize, 1))
        # flips output left/right with respect to worm's body
        self.IllumFlipLR = np.zeros((framessize, 1))
        # center of the freehand rectangular illumination in wormspace
        self.IllumRectOrigin = np.zeros((framessize, 2))
        # xy value describing dimension of rectangle
        self.IllumRectRadius = np.zeros((framessize, 2))
        # velocity sent to stage in stage units/second
        self.StageVelocity = np.zeros((framessize, 2))
        self.StagePosition = np.zeros((framessize, 2))
        self.StageFeedbackTarget = np.zeros((framessize, 2))
        self.FirstLaser = np.zeros((framessize, 1))
        self.SecondLaser = np.zeros((framessize, 1))
        self.ProtocolIsOn = np.zeros((framessize, 1))
        self.ProtocolStep = np.zeros((framessize, 1))

        self.curve_data = np.zeros((framessize, 100))  # curve_data
        self.angle_data = np.zeros((framessize, 101))  # angle_data
        self.curvedatafiltered = np.zeros(
            (framessize, 100))  # curvedatafiltered
        # self.LaserPower = []  # 一个空的list
        # self.GreenLaser = np.zeros((framessize,1))  #int 0-100 of relative laser power. -1 means leaser is not being controlled programmatically
        # self.BlueLaser = np.zeros((framessize,1)) #int 0-100 of relative laser power. -1 means leaser is not being controlled programmatically
        # self.HeadCurv = np.zeros((framessize,1)) #curvature of the head
        # self.HeadCurvDeriv = np.zeros((framessize,1)) #derivative of curvature of the head


In [ ]:
def Get_ExperimentTime(lines:list):
    # 获取'ExperimentTime: '
    import re
    for i in range(20):
        if re.match('ExperimentTime',lines[i]):
            break
    str0 = lines[i].split('ExperimentTime')
    str0 = str0[1].strip(':') # 去除冒号
    str0 = str0.replace('"',"") # 去除引号 ""
    str0 = str0.replace('\\n',"") # 删除末尾的回车
    ExperimentTime = str0
    return(ExperimentTime)

def Get_DefaultGrid(lines:list):
    import numpy as np
    linenumber = Get_Frames_line(lines)-2
    data = np.zeros((1,2))
    data[0,0] = int(lines[linenumber].split(':')[-1])
    data[0,1] = int(lines[linenumber+1].split(':')[-1])
    return(data)


In [ ]:
def Get_One_Frame(lines,line_start):
    # 获得一帧的所有数据进入一个list
    # lines是读取的yaml文件,是一个list
    # line_start是这一帧开始的行
    OneFrameList = []
    for i in range(line_start,line_start+98):
        OneFrameList.append(lines[i])
    return(OneFrameList)

def Get_Any_Frame(lines,Frames_line,number):
    # 获取任意指定帧的数据
    # 从第1帧开始数
    begin_num = Get_First_Frames(lines)
    end_num = Get_End_Frames(lines)
    All_Frames_num = end_num-begin_num+1  # 总的帧数
    if number < 1 or number>All_Frames_num:
        print("please input:[",1,",",All_Frames_num,"]")
        return(0)
    else:
        frames_line_num = Frames_line+2+(number-1)*99
        return(Get_One_Frame(lines,frames_line_num))

framesline = Get_Frames_line(ListFile)  # Frames 的行数

In [ ]:
def ConvertData(lists:list,begin_data:int,end_data:int):
    import numpy as np

    # 将begin_data~end_data 连起来变为 2*100的数组
    data0 = []
    for i in range(begin_data,end_data+1):
        data0.append(lists[i])
    data0 = ''.join(data0)
    data = data0[8:-2] # 去除前面的和后面的字符
    data = list(map(int,data.split(',')))  # 将list中的字符串转为int
    data = np.array(data)
    data0 = np.zeros((2,100))
    data0[0,:] = data[0:200:2]  # x隔一个取一个值
    data0[1,:] = data[1:200:2]  # y隔一个取一个值
    return(data0)

In [ ]:

class Extract_OneFrame(object):
    # 提取一帧的 类
    def __init__(self,OneFrameList:list):
        import numpy as np
        self.FrameNumber = int(OneFrameList[0].split(':')[-1])
        self.TimeElapsed = int(OneFrameList[1].split(':')[-1])+0.001*float(OneFrameList[2].split(':')[-1])  # 时间
        self.Head = np.array([int(OneFrameList[4].split(':')[-1]),int(OneFrameList[5].split(':')[-1])])
        self.Tail = np.array([int(OneFrameList[7].split(':')[-1]),int(OneFrameList[8].split(':')[-1])])

        self.BoundaryA = ConvertData(OneFrameList,13,29)

        self.BoundaryB = ConvertData(OneFrameList,34,50)

        self.Centerline = ConvertData(OneFrameList,55,71)

        self.DLPisOn =  int(OneFrameList[72].split(':')[-1])
        self.FloodLightIsOn = int(OneFrameList[73].split(':')[-1])
        self.IllumInvert = int(OneFrameList[74].split(':')[-1])
        self.IllumFlipLR = int(OneFrameList[75].split(':')[-1])
        self.IllumRectOrigin = np.array([int(OneFrameList[77].split(':')[-1]),int(OneFrameList[78].split(':')[-1])])
        self.IllumRectRadius = np.array([int(OneFrameList[80].split(':')[-1]),int(OneFrameList[81].split(':')[-1])])
        self.StageVelocity = np.array([int(OneFrameList[83].split(':')[-1]),int(OneFrameList[84].split(':')[-1])])
        self.StagePosition = np.array([int(OneFrameList[86].split(':')[-1]),int(OneFrameList[87].split(':')[-1])])
        self.StageFeedbackTarget = np.array([int(OneFrameList[91].split(':')[-1]),int(OneFrameList[92].split(':')[-1])])
        self.FirstLaser = int(OneFrameList[94].split(':')[-1])
        self.SecondLaser = int(OneFrameList[95].split(':')[-1])
        self.ProtocolIsOn = int(OneFrameList[96].split(':')[-1])
        self.ProtocolStep = int(OneFrameList[97].split(':')[-1])
        # self.curve_data # curve_data
        # self.angle_data   # angle_data
        # self.curvedatafiltered  curvedatafiltered
        
        # self.LaserPower = OneFrameList[93].split(':')[-1]
        # self.GreenLaser = 
        # self.BlueLaser =
        # self.HeadCurv = 
        # self.HeadCurvDeriv =

In [ ]:
 Get_Frames_line(ListFile)+2+(19844-1)*99

In [ ]:
len(ListFile)

In [ ]:
YamlFiles = YamlFrames(wormname, All_Frames_num)
YamlFiles.ExperimentTime = Get_ExperimentTime(ListFile)
print(YamlFiles.ExperimentTime)

YamlFiles.DefaultGridSizeForNonProtocolIllum = Get_DefaultGrid(ListFile)



for i in range(0, int(real_frames)):
    print(i)
    framelist = Get_Any_Frame(ListFile,framesline,i+1)  # 提取的一帧的内容
    frame = Extract_OneFrame(framelist)
    # internal frame number, not nth recorded frame
    YamlFiles.FrameNumber[i, :] = frame.FrameNumber
    # time since start of experiment (in s) = sElapsed+ 0.001*msRemElapsed
    YamlFiles.TimeElapsed[i, :] = frame.TimeElapsed
    # N*2*100 x,y BoundaryA position in pixels on camera
    YamlFiles.BoundaryA[i, 0, :] = frame.BoundaryA[0, :]
    YamlFiles.BoundaryA[i, 1, :] = frame.BoundaryA[1, :]

    # N*2*100 x,y BoundaryB position in pixels on camera
    YamlFiles.BoundaryB[i, 0, :] = frame.BoundaryB[0, :]
    YamlFiles.BoundaryB[i, 1, :] = frame.BoundaryB[1, :]

    # N*2*100 x,y centerline position in pixels on camera
    YamlFiles.Centerline[i, 0, :] = frame.Centerline[0, :]
    YamlFiles.Centerline[i, 1, :] = frame.Centerline[1, :]

    YamlFiles.Head[i, :] = frame.Head[:]  # position in pixels on camera
    YamlFiles.Tail[i, :] = frame.Tail[:]  # position in pixels on camera
    YamlFiles.DLPisOn[i] = frame.DLPisOn  # bool whether DLP is active

    # flood light overrides all other patterns and hits entire fov
    YamlFiles.FloodLightIsOn[i] = frame.FloodLightIsOn
    # whether pattern is inverted (invert has precedence over floodlight)
    YamlFiles.IllumInvert[i] = frame.IllumInvert
    # flips output left/right with respect to worm's body
    YamlFiles.IllumFlipLR[i] = frame.IllumFlipLR
    # center of the freehand rectangular illumination in wormspace
    YamlFiles.IllumRectOrigin[i, :] = frame.IllumRectOrigin[:]
    # xy value describing dimension of rectangle
    YamlFiles.IllumRectRadius[i, :] = frame.IllumRectRadius[:]
    # velocity sent to stage in stage units/second
    YamlFiles.StageVelocity[i, :] = frame.StageVelocity[:]
    YamlFiles.StagePosition[i, :] = frame.StagePosition[:]
    YamlFiles.StageFeedbackTarget[i, :] = frame.StageFeedbackTarget[:]
    YamlFiles.FirstLaser[i] = frame.FirstLaser
    YamlFiles.SecondLaser[i] = frame.SecondLaser
    YamlFiles.ProtocolIsOn[i] = frame.ProtocolIsOn
    YamlFiles.ProtocolStep[i] = frame.ProtocolStep


In [ ]:
print(7529+19843)
19843*98

In [ ]:
centerline = YamlFiles.Centerline